In [54]:
import tools
import datetime
import numpy as np

In [156]:
df = tools.load_dataframe()

In [3]:
df.head()

,DAY,HOUR,OPCAT_CATEGORY,SIZE,SUBBRANCH_ID
0,2014-10-25,16,SM,26,1005609
1,2014-04-14,11,SM,21,1005625
2,2014-10-25,11,SM,36,1005609
3,2015-08-14,12,CM,175,1006034
4,2014-04-14,12,SM,29,1005625


# Baseline: predict average value

In [226]:
dates = [
    datetime.datetime.strptime("01.03.2015", "%d.%m.%Y").date(),
    datetime.datetime.strptime("01.04.2015", "%d.%m.%Y").date(),
    datetime.datetime.strptime("01.08.2015", "%d.%m.%Y").date(),
    datetime.datetime.strptime("01.09.2015", "%d.%m.%Y").date(),
    datetime.datetime.strptime("01.10.2015", "%d.%m.%Y").date(),
]
def filter_standart_work_hours(df, start_time=8, end_time=19):
    return df[(df["HOUR"] >= start_time) & ((df["HOUR"] <= end_time))]

def filter_category(df, category):
    return df[(df["OPCAT_CATEGORY"] == category)]

def make_train_test(df, bounder_data, acc, delta=datetime.timedelta(days=31), short_history=None):    
    train = df[df["DAY"] < bounder_data]
    test = df[(df["DAY"] > bounder_data) & (df["DAY"] > bounder_data + delta)]
    
    if short_history:
        train = train[train["DAY"] > bounder_data - short_history*delta]

    train = filter_standart_work_hours(train)
    test = filter_standart_work_hours(test)
    
    for category in ["SM", "CM"]:
        
        train_ = filter_category(train, category)
        test_ = filter_category(test, category)
        
        train_model = train_.groupby(["SUBBRANCH_ID", "HOUR"])["SIZE"].mean()
        test_model = test_.groupby(["SUBBRANCH_ID", "HOUR"])["SIZE"].mean()
        acc_ = ((train_model - test_model)**2).mean()
        acc[category].append(acc_)
    return acc

acc = {
    'SM' : [],
    'CM' : [],
}
for data in dates:
    make_train_test(df, data, acc)
print acc
print 'SM', np.array(acc['SM']).mean(), np.array(acc['SM']).std()
print 'CM', np.array(acc['CM']).mean(), np.array(acc['CM']).std()

{'CM': [170.2016107967591, 171.09478338916668, 147.26514349942343, 157.77679956832114, 166.43012971757324], 'SM': [28.927186323232775, 26.670184082414401, 12.396111273763523, 12.87045128304009, 11.701012612561552]}
SM 18.512989115 7.62433977218
CM 162.553693394 8.97804087735


# Baseline: predict average without new year holidays

In [227]:
newyear_2014 = [
    datetime.datetime.strptime("15.12.2014", "%d.%m.%Y").date(),
    datetime.datetime.strptime("15.01.2015", "%d.%m.%Y").date()
]
newyear_2015 =  [
    datetime.datetime.strptime("15.12.2015", "%d.%m.%Y").date(),
    datetime.datetime.strptime("12.01.2016", "%d.%m.%Y").date()
]

In [228]:
def remove_interval(df, holidays):
    return df[(df["DAY"] < holidays[0]) | (df["DAY"] > holidays[1])]

df_ordinal = remove_interval(df, newyear_2014)
df_ordinal = remove_interval(df_ordinal, newyear_2015)

acc = {
    'SM' : [],
    'CM' : [],
}
for data in dates:
    make_train_test(df_ordinal, data, acc)
print acc
print 'SM', np.array(acc['SM']).mean(), np.array(acc['SM']).std()
print 'CM', np.array(acc['CM']).mean(), np.array(acc['CM']).std()

{'CM': [168.50155010822337, 167.79878331851862, 140.89994313931282, 146.76465133036211, 143.43209413721539], 'SM': [30.569068920005886, 28.501864873757352, 13.820398619673984, 14.647363940410941, 13.899756295377438]}
SM 20.2876905298 7.58451388323
CM 153.479404407 12.1242622225


# Baseline: predict average without new year holidays (N month only)

In [231]:
df_ordinal = remove_interval(df, newyear_2014)
df_ordinal = remove_interval(df_ordinal, newyear_2015)

acc = {
    'SM' : [],
    'CM' : [],
}
for data in dates:
    make_train_test(df_ordinal, data, acc, datetime.timedelta(days=31), 1)
print acc
print 'SM', np.array(acc['SM']).mean(), np.array(acc['SM']).std()
print 'CM', np.array(acc['CM']).mean(), np.array(acc['CM']).std()

{'CM': [139.72939806550067, 270.73378123811983, 141.32694080352383, 155.41342687979949, 87.425452232901449], 'SM': [7.8268431794109157, 10.770094378867448, 12.447775212500309, 9.7047473962756783, 8.0721867132486835]}
SM 9.76432937606 1.72239335599
CM 158.925799844 60.5043516725


In [233]:
for data in dates:
    make_train_test(df_ordinal, data, acc, datetime.timedelta(days=31), 3)
print acc
print 'SM', np.array(acc['SM']).mean(), np.array(acc['SM']).std()
print 'CM', np.array(acc['CM']).mean(), np.array(acc['CM']).std()

{'CM': [139.72939806550067, 270.73378123811983, 141.32694080352383, 155.41342687979949, 87.425452232901449, 197.35012628960521, 194.46099370907302, 72.884898250389597, 165.65448509844026, 115.61931747601984, 251.4894419456308, 222.33998064360892, 81.896916512022898, 109.45483933731985, 135.36385109749349], 'SM': [7.8268431794109157, 10.770094378867448, 12.447775212500309, 9.7047473962756783, 8.0721867132486835, 6.7587451208097944, 7.5332282298020727, 9.0115496745753561, 10.737408234363725, 7.5702581925976675, 8.7618768466393799, 6.6724619199678603, 9.4072715776006994, 10.059964873595996, 8.5029387714007889]}
SM 8.92249002144 1.56869800232
CM 156.076256639 58.748278294


In [234]:
for data in dates:
    make_train_test(df_ordinal, data, acc, datetime.timedelta(days=31), 6)
print acc
print 'SM', np.array(acc['SM']).mean(), np.array(acc['SM']).std()
print 'CM', np.array(acc['CM']).mean(), np.array(acc['CM']).std()

{'CM': [139.72939806550067, 270.73378123811983, 141.32694080352383, 155.41342687979949, 87.425452232901449, 197.35012628960521, 194.46099370907302, 72.884898250389597, 165.65448509844026, 115.61931747601984, 251.4894419456308, 222.33998064360892, 81.896916512022898, 109.45483933731985, 135.36385109749349, 279.8218629858244, 312.75891936618501, 90.455195447044915, 103.99805956895032, 117.78286103296975], 'SM': [7.8268431794109157, 10.770094378867448, 12.447775212500309, 9.7047473962756783, 8.0721867132486835, 6.7587451208097944, 7.5332282298020727, 9.0115496745753561, 10.737408234363725, 7.5702581925976675, 8.7618768466393799, 6.6724619199678603, 9.4072715776006994, 10.059964873595996, 8.5029387714007889, 17.407938911704544, 10.789032847132058, 5.7744071387889901, 9.3632445725736737, 8.8088975227422175]}
SM 9.29904356573 2.44622943076
CM 162.298037399 70.4781066146


# Linear solution

In [235]:
from sklearn import linear_model

def test_linear_solution(df, bounder_data, acc, delta=datetime.timedelta(days=31), short_history=None):    
    train = df[df["DAY"] < bounder_data]
    test = df[(df["DAY"] > bounder_data) & (df["DAY"] > bounder_data + delta)]
    
    if short_history:
        train = train[train["DAY"] > bounder_data - short_history*delta]

    train = filter_standart_work_hours(train)
    test = filter_standart_work_hours(test)

    for category in ["SM", "CM"]:
        train_ = filter_category(train, category)
        test_ = filter_category(test, category)
        
        train_model = train_.groupby(["SUBBRANCH_ID", "HOUR"])
        test_model = test_.groupby(["SUBBRANCH_ID", "HOUR"])["SIZE"].mean()
        
        errors = []
        
        for group in train_model.groups.keys():
            x = train_model.get_group(group)["DAY"]
            y = train_model.get_group(group)["SIZE"]
            timestamps = np.array([int(x_.strftime("%s")) for x_ in x.values])
            reg = linear_model.LinearRegression(copy_X=True)
            reg.fit(timestamps.reshape((len(timestamps)), 1), y.values)
            prediction = reg.predict(int((bounder_data+delta/2).strftime("%s")))[0]
            errors.append(prediction - test_model[group[0]][group[1]])
            
        acc_ = (np.array(errors)**2).mean()
        acc[category].append(acc_)
    return acc

In [222]:
df_ordinal = remove_interval(df, newyear_2014)
df_ordinal = remove_interval(df_ordinal, newyear_2015)

acc = {
    'SM' : [],
    'CM' : [],
}
for data in dates:
    test_linear_solution(df_ordinal, data, acc)
print acc
print 'SM', np.array(acc['SM']).mean(), np.array(acc['SM']).std()
print 'CM', np.array(acc['CM']).mean(), np.array(acc['CM']).std()

{'CM': [326.19279588293244, 371.88835099204567, 186.19976973307223, 167.54088522361718, 181.07338765633079], 'SM': [17.866294335031018, 11.532559194670961, 5.2217357432548601, 8.2885844373642303, 8.7684745105031912]}
SM 10.3355296442 4.26478208368
CM 246.579037898 85.1169198509


# Polinom solution

In [217]:
from sklearn import linear_model

def test_polinom_solution(df, bounder_data, acc, delta=datetime.timedelta(days=31)):    
    train = df[df["DAY"] < bounder_data]
    test = df[(df["DAY"] > bounder_data) & (df["DAY"] > bounder_data + delta)]

    train = filter_standart_work_hours(train)
    test = filter_standart_work_hours(test)

    for category in ["SM", "CM"]:
        train_ = filter_category(train, category)
        test_ = filter_category(test, category)
        
        train_model = train_.groupby(["SUBBRANCH_ID", "HOUR"])
        test_model = test_.groupby(["SUBBRANCH_ID", "HOUR"])["SIZE"].mean()
        
        errors = []
        
        for group in train_model.groups.keys():
            x = train_model.get_group(group)["DAY"]
            y = train_model.get_group(group)["SIZE"]
            timestamps = np.array([[int(x_.strftime("%s")), int(x_.strftime("%s"))**2] for x_ in x.values])
            reg = linear_model.LinearRegression(copy_X=True)
            reg.fit(timestamps, y.values)
            prediction_time = int((bounder_data+delta/2).strftime("%s"))
            prediction = reg.predict(np.array([prediction_time, prediction_time**2]).reshape(1, 2))[0]
            errors.append(prediction - test_model[group[0]][group[1]])
            
        acc_ = (np.array(errors)**2).mean()
        acc[category].append(acc_)
    return acc

In [218]:
df_ordinal = remove_interval(df, newyear_2014)
df_ordinal = remove_interval(df_ordinal, newyear_2015)

acc = {
    'SM' : [],
    'CM' : [],
}
for data in dates:
    test_polinom_solution(df_ordinal, data, acc)
print acc
print 'SM', np.array(acc['SM']).mean(), np.array(acc['SM']).std()
print 'CM', np.array(acc['CM']).mean(), np.array(acc['CM']).std()

{'CM': [395.11770324535104, 411.40196542172617, 128.72935265113867, 187.46105547006798, 206.76994028884781], 'SM': [5.8797842223553856, 7.9421714896536173, 25.794484693366975, 30.174496031609845, 26.545579429371386]}
SM 19.2673031733 10.2179110509
CM 265.896003415 115.180601811


# Linear solution (last N month only)

In [236]:
df_ordinal = remove_interval(df, newyear_2014)
df_ordinal = remove_interval(df_ordinal, newyear_2015)

acc = {
    'SM' : [],
    'CM' : [],
}
for data in dates:
    test_linear_solution(df_ordinal, data, acc, datetime.timedelta(days=31), 1)
print acc
print 'SM', np.array(acc['SM']).mean(), np.array(acc['SM']).std()
print 'CM', np.array(acc['CM']).mean(), np.array(acc['CM']).std()

{'CM': [294.28338809504055, 749.28610002485209, 1422.4493708977743, 1283.5140711899248, 470.32002316619736], 'SM': [46.537951342551331, 47.370254527755343, 50.437149822562027, 45.645733584405562, 31.505039815365656]}
SM 44.2992258185 6.5972948855
CM 843.970590675 442.396222968


In [238]:
df_ordinal = remove_interval(df, newyear_2014)
df_ordinal = remove_interval(df_ordinal, newyear_2015)

acc = {
    'SM' : [],
    'CM' : [],
}
for data in dates:
    test_linear_solution(df_ordinal, data, acc, datetime.timedelta(days=31), 3)
print acc
print 'SM', np.array(acc['SM']).mean(), np.array(acc['SM']).std()
print 'CM', np.array(acc['CM']).mean(), np.array(acc['CM']).std()

{'CM': [202.29683477283496, 195.4583468397214, 239.26460249162955, 532.75981270231637, 97.444140835391224], 'SM': [9.2001962714986778, 21.368292852461312, 14.940754483419896, 15.320321287652343, 10.668352529352861]}
SM 14.2995834849 4.25697653832
CM 253.444747528 147.331397251


In [239]:
df_ordinal = remove_interval(df, newyear_2014)
df_ordinal = remove_interval(df_ordinal, newyear_2015)

acc = {
    'SM' : [],
    'CM' : [],
}
for data in dates:
    test_linear_solution(df_ordinal, data, acc, datetime.timedelta(days=31), 6)
print acc
print 'SM', np.array(acc['SM']).mean(), np.array(acc['SM']).std()
print 'CM', np.array(acc['CM']).mean(), np.array(acc['CM']).std()

{'CM': [350.03659339883257, 235.4641634471767, 138.49233353154571, 296.52056559276645, 172.58071392366904], 'SM': [9.6080427912539594, 11.540122345730904, 16.889308495169519, 13.767693496723847, 8.5305996495243956]}
SM 12.0671533557 2.99924042153
CM 238.618873979 77.6695937629
